In [1]:
# import all necessary libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)

In [3]:
toronto = pd.DataFrame()
soup = BeautifulSoup(page.text, 'lxml')
wikitable = soup.find_all('table')[0] 
row_marker = 0
for row in wikitable.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        toronto.loc[row_marker,column_marker] = column.get_text()
        column_marker += 1
    row_marker += 1

toronto.rename(columns={0:'PostalCode',1:'Borough',2:'Neighborhood'}, inplace=True)

toronto = toronto[toronto.Borough!='Not assigned']
toronto.reset_index(drop=True,inplace=True)
toronto_n= pd.DataFrame(columns = ['PostalCode','Borough','Neighborhood'])
for ii in range(toronto.shape[0]):
    borough = toronto.loc[ii,'Borough']
    postalcode = toronto.loc[ii,'PostalCode']
    neighborhood = toronto.loc[ii,'Neighborhood'][:-1]
    if neighborhood == 'Not assigned':
        neighborhood = borough
    
    if postalcode in toronto_n.PostalCode.values:
        old_neighborhood = toronto_n[toronto_n.PostalCode == postalcode].Neighborhood
        new_value = (old_neighborhood + ',' + neighborhood)
        toronto_n.loc[toronto_n.PostalCode == postalcode,'Neighborhood'] = new_value 
    
    else:
        toronto_n = toronto_n.append({'PostalCode': postalcode,
                                                'Borough': borough,
                                                'Neighborhood': neighborhood,
                                               }, ignore_index=True)

toronto_n[['PostalCode','waste']] = toronto_n.PostalCode.str.split("\n",expand=True)
toronto_n[['Borough','waste']] = toronto_n.Borough.str.split("\n",expand=True)
toronto_n[['Neighborhood','waste']] = toronto_n.Neighborhood.str.split("\n",expand=True)
toronto_n.drop(['waste'],axis=1,inplace=True)

In [4]:
toronto_n.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
toronto_n.shape

(180, 3)

# Question:2

In [26]:
geospatial = pd.read_csv('Geospatial_Coordinates.csv')

for postalcode in toronto_n['PostalCode']:
    latitude = geospatial.loc[toronto_n.PostalCode == postalcode,'Latitude']
    longitude = geospatial.loc[toronto_n.PostalCode == postalcode,'Longitude']    
    toronto_n.loc[toronto_n.PostalCode == postalcode,'Latitude'] =  latitude 
    toronto_n.loc[toronto_n.PostalCode == postalcode,'Longitude'] =  longitude


In [43]:
toronto_n.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A,Not assigned,Not assigned,43.806686,-79.194353
1,M2A,Not assigned,Not assigned,43.784535,-79.160497
2,M3A,North York,Parkwoods,43.763573,-79.188711
3,M4A,North York,Victoria Village,43.770992,-79.216917
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.773136,-79.239476


## Using geopy library to get the latitude and longitude values of New York City.

# Question:3

In [20]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="To_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [30]:
data=toronto_n.copy(deep=True)
data.dropna(inplace=True)

In [35]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [42]:
for lat, lng, borough, neighborhood in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto